In [ ]:
from utils.simulation.sim_utils import RandomSimulator
from utils.core.save_manager import SaveUtils
from utils.data_io import load_data
import os
from utils.viz.general_viz import Visualisation
from utils.eda.correlation import CorrelationAnalyzer
import pandas as pd
import numpy as np
from utils.simulation.decomposition import Decomposer
from utils.simulation.covariance_matrix import CovarianceMatrix
from utils.simulation.monte_carlo_simulator import MonteCarloSimulator

🧪 Simulating Unscaled Normally Distributed Data with Specified Skewness and Kurtosis

In [ ]:
custom_column_names = [
    "Interest_Rate", "Loan_Amount", "Employment_Length",
    "Age", "Income", "Credit_Score",
    "Debt_To_Income", "Home_Ownership", "Purpose", "Region"
]

simulator = RandomSimulator(num_simulations=1000,column_names=custom_column_names)
df = simulator.simulate_normal(num_variables=10, target_skew=0, target_kurt=3)

save_util = SaveUtils()
save_util.save_dataframe_to_csv(df,os.path.join(os.getcwd(), "data/output/simulated_normal.csv"), overwrite=True)

🧪 Simulate scaled normals (e.g., volatilities)

In [ ]:
# params = [0.2, 0.15, 0.3]  # std devs or volatilities
# simulator = RandomSimulator(parameters=params)
# df = simulator.simulate_normal(target_skew=0, target_kurt=3)

📊 Simulating Poisson Data from Excel Input
- Reads input data from an Excel file
- Initializes a Poisson simulator
- Simulates Poisson-distributed values
- Saves the simulated data to an Excel file

In [ ]:
df_input  = load_data(
    source_type='excel',
    input_path= os.path.join(os.getcwd(), "data/input/Poisson Simulation.xlsx"),
    sheet_name='Lambda Calculation',
    usecols=['Class','Lambda']
)
parameters = df_input["Lambda"].values
column_names = [f"Class_{int(c)}" for c in df_input["Class"].values]

simulator = RandomSimulator(parameters=parameters, column_names=column_names,num_simulations = 10000)
sim = simulator.simulate_poisson()
df_poisson = sim.round().astype(int)
save_util.save_dataframe_to_excel(
    df_poisson,
    os.path.join(os.getcwd(), "data/output/simulated_poisson.xlsx"),
    sheet_name="simulated values", 
    overwrite=True
    )


#### 🔗 2.5 Examine Variable Correlations

This section calculates and displays correlations between different types of variables in the `main_df`.

- **`num_method` (str)**: Defines the method for calculating correlation between numerical variables. Allowed values are:
    - `'pearson'` *(default)*: Standard Pearson linear correlation coefficient.
    - `'spearman'`: Spearman's rank correlation coefficient (for monotonic relationships).
    - `'kendall'`: Kendall's tau correlation coefficient (for ordinal or non-normally distributed data).

- **`cat_method` (str)**: Defines the method for calculating association between categorical variables. Allowed values are:
    - `'cramers_v'` *(default)*: Cramer's V (measures association between nominal categorical variables).
    - `'mutual_info'`: Mutual Information (measures the statistical dependence between two random variables).

- **`cat_num_method` (str)**: Defines the method for calculating association between categorical and numerical variables. Allowed values are:
    - `'correlation_ratio'` *(default)*: Correlation Ratio (Eta squared, measures variance explained).
    - `'f_test'`: F-statistic from ANOVA (assesses the difference in means across categories).
    - `'mutual_info'`: Mutual Information (measures the statistical dependence). 
    - `'kruskal'`: Non-parametric alternative to ANOVA. Compares distributions of a continuous variable across categories. Good when your numerical variables are not normally distributed
    - `'target_spearman'`: Replaces each category with the mean of the target variable (e.g. default rate). Then computes correlation with numerical features. Captures ordinal structure or monotonic trends across groups

In [ ]:
main_df = load_data(
    source_type='csv',
    input_path= os.path.join(os.getcwd(), "data/input/returns_raw.csv"),
)

In [ ]:
analyzer = CorrelationAnalyzer(main_df)
corr_df,corr_matrix = analyzer.correlation_matrix(num_method="pearson", cat_method="cramers_v",
                                      cat_num_method="correlation_ratio",return_matrix=True)

Visualisation.plot_heatmap_matrix(corr_matrix, title="Correlation Matrix")

CovarianceMatrix
----------------
A utility class for calculating the covariance matrix of asset returns,
optionally annualized. Intended for use in financial simulations such as
Monte Carlo modeling.

In [ ]:
cov_calc = CovarianceMatrix(main_df)
cov_matrix = cov_calc.get_matrix()
# print(cov_matrix)

#### Cholesky Decomposition and Visualisation

We perform a Cholesky decomposition on the correlation matrix to obtain a lower triangular matrix.  
This decomposition is useful for simulations and generating correlated random variables.  
We then visualise the resulting matrix using a heatmap for better interpretation.

In [ ]:
Decomposition_df = Decomposer.cholesky_decomposition(cov_matrix)
# Visualisation.plot_heatmap_matrix(Decomposition_df, title="Cholesky Decomposition Matrix")


##### 📊 Monte Carlo Simulation Using Rubinstein's Approach

In [ ]:
# main_df = load_data(
#     source_type='csv',
#     input_path= os.path.join(os.getcwd(), "data/input/Monte Carlo Multivariable.csv"),
# )
main_df = load_data(
    source_type='csv',
    input_path= os.path.join(os.getcwd(), "data/input/Monte Carlo Univariable.csv"),
)

sim = MonteCarloSimulator(main_df,num_simulations=10000)
sim.run_simulation()
multivariate_MC_simulation = sim.get_final_simulated_values()
# covariance_matrix = sim.get_covariance_matrix()
raw_normal_simulation = sim.get_raw_simulations()
# cholesky_matrix = sim.get_cholesky_matrix()

save_util.save_dataframe_to_csv(
    multivariate_MC_simulation,
    os.path.join(os.getcwd(), "data/output/MonteCarlo final_sim_u.csv"),
    overwrite=True
    )

save_util.save_dataframe_to_csv(
    raw_normal_simulation,
    os.path.join(os.getcwd(), "data/output/MonteCarlo random_sim_u.csv"),
    overwrite=True
    )

# save_util.save_dataframe_to_csv(
#     covariance_matrix,
#     os.path.join(os.getcwd(), "data/output/MonteCarlo cov_matrix.csv"),
#     overwrite=True
#     )


# save_util.save_dataframe_to_csv(
#     cholesky_matrix,
#     os.path.join(os.getcwd(), "data/output/MonteCarlo cholesky_matrix.csv"),
#     overwrite=True
#     )

Reading From a Json Source

In [ ]:
from utils.data_io import load_data
api_data = '[["name", "age"], ["Alice", 30], ["Bob", 25]]'
df = load_data(source_type='json', json_source=api_data)
print(df)

Fitting **Beta** distributions and then simulating random numbers based on the result

In [ ]:
from utils.eda.statistical import StatisticalAnalysis
from utils.simulation.sim_utils import RandomSimulator
import pandas as pd
import numpy as np


# Step 1: Create sample dataframe with random numbers between 0 and 1
np.random.seed(42)  # for reproducibility
df = pd.DataFrame({
    'default_rate': np.random.rand(100)
})


# Step 2: Initialize StatisticalAnalysis
stat = StatisticalAnalysis(df)

# Step 3: Fit only 'beta' and 'logit-normal'
distribution_results = stat.fit_best_distribution(
    ['default_rate'],
    method='sumsquare_error',
    common_distributions=False,
    distribution_list=['beta'],  # assuming 'logitnorm' works with Fitter
    timeout=300
)

print("\nReturned Results:", distribution_results)


params = distribution_results['default_rate']['parameters']
simulator = RandomSimulator(
    parameters=[(params['a'], params['b'], params['loc'], params['scale'])],
    num_simulations=10000,
    column_names=['default_rate_sim']
)
df_sim = simulator.simulate_beta()
print(df_sim.head())

In [ ]:
from utils.eda.statistical import StatisticalAnalysis
from utils.simulation.sim_utils import RandomSimulator
import pandas as pd
import numpy as np

# Step 1: Create sample dataframe
np.random.seed(42)
df = pd.DataFrame({
    'default_rate': np.random.randint(100, 1001, size=100)  # 100 integers from 100 to 1000 inclusive
})

# Step 2: Initialize StatisticalAnalysis
stat = StatisticalAnalysis(df)

# Step 3: Fit only 'logit-normal' and 'gamma'
distribution_results = stat.fit_best_distribution(
    ['default_rate'],
    method='sumsquare_error',
    common_distributions=False,
    distribution_list=['lognorm', 'gamma'],
    timeout=300
)

print("\nReturned Results:", distribution_results)

# Step 4: Loop through results and simulate using the winning distribution
for col, res in distribution_results.items():
    best_dist = res['best_distribution']   # e.g., 'lognorm' or 'gamma'
    params = res['parameters']

    simulator = RandomSimulator(
        parameters=[tuple(params.values())],  # convert dict to tuple
        num_simulations=10000,
        column_names=[f"{col}_sim"]
    )

    if best_dist == 'lognorm':
        ordered_params = (params['s'], params['loc'], params['scale'])
        simulator = RandomSimulator(
            parameters=[ordered_params],
            num_simulations=10000,
            column_names=[f"{col}_sim"]
        )
        df_sim = simulator.simulate_lognormal()
    elif best_dist == 'gamma':
        ordered_params = (params['a'], params['loc'], params['scale'])
        simulator = RandomSimulator(
            parameters=[ordered_params],
            num_simulations=10000,
            column_names=[f"{col}_sim"]
        )
        df_sim = simulator.simulate_gamma()
    else:
        raise ValueError(f"Unsupported distribution: {best_dist}")

    print(f"\nSimulation for {col} ({best_dist}):")
    print(df_sim.head())